In [ ]:
var('z t u v alpha')
var('r d', domain='integer')

In [ ]:
SR_restr = SR.subring(accepting_variables=(r, alpha, d))
SCR = SR.subring(no_variables=True)
A.<n> = AsymptoticRing('QQ^n * n^QQ', SCR)
A_func.<Z> = AsymptoticRing('Z^QQ * log(Z)^QQ', SR_restr)

In [ ]:
def T(z,t):
    return 1/2 * (1 - (z-t) - sqrt(1 - 2*(z+t) + (z-t)^2))

def upsilon(z):
    return (1 - sqrt(1 - 4*z))/(2*z) - 1

def chi(k):
    return 2*pi*I*k/log(2)

In [ ]:
T(z,t).taylor((z, 0), (t, 0), 10)

In [ ]:
show(T(z, z*t).diff(t).subs(t=1))

In [ ]:
show(T(z, z*t).diff(t, t).subs(t=1).factor())

In [ ]:
catalan_shifted_asy = asymptotic_expansions.Binomial_kn_over_n(n, k=2, precision=6).subs(n=n-1)/n
catalan_shifted_asy.show()

## Cutting of Leaves

In [ ]:
def G_r(u, v):
    return (1 - u^(r+2))/((1 - u^(r+1))*(1 + u)) * \
            T(u*(1-u^(r+1))^2/(1 - u^(r+2))^2 * v, u^(r+1)*(1-u)^2/(1 - u^(r+2))^2 *v)

Expansion of generating function at $z = 1/4$ in $z^-1$ (read $Z = (1-4z)^{-1}$):

In [ ]:
z_asy = 1/4*(1-Z^-1)
u_asy = upsilon(z_asy)+O(Z^-3); u_asy

In [ ]:
u_asy_r = u_asy^r; u_asy_r

In [ ]:
show(1-u_asy)

In [ ]:
r_rules = {1/sqrt((r^2+2*r+1)/(r+2)^2): (r+2)/(r+1),
          ((r^2+2*r+1)/(r+2)^2)^(-3/2): ((r+2)/(r+1))^3}

In [ ]:
L_expectation_expansion = fast_callable(G_r(u,v).diff(v,1)(v=1), vars=(u, r))(u_asy, r)
L_expectation_expansion = L_expectation_expansion.map_coefficients(lambda ex: ex.factor().subs(r_rules))
L_expectation_expansion.show()

In [ ]:
L_expectation_asy = L_expectation_expansion._singularity_analysis_(n, zeta=1/4, precision=5) / catalan_shifted_asy
L_expectation_asy = L_expectation_asy.map_coefficients(factor)
L_expectation_asy.show()

In [ ]:
L_fmoment2_expansion = fast_callable(G_r(u,v).diff(v, 2)(v=1), vars=(u, r))(u_asy, r)
L_fmoment2_expansion = L_fmoment2_expansion.map_coefficients(lambda ex: ex.factor().subs(r_rules))
L_fmoment2_expansion.show()

In [ ]:
L_fmoment2_asy = L_fmoment2_expansion._singularity_analysis_(n, zeta=1/4, precision=5) / catalan_shifted_asy
L_fmoment2_asy = L_fmoment2_asy.map_coefficients(factor)
L_fmoment2_asy.show()

In [ ]:
L_variance_asy = L_fmoment2_asy + L_expectation_asy - L_expectation_asy^2
L_variance_asy = L_variance_asy.map_coefficients(factor)
L_variance_asy.show()

In [ ]:
# first three derivatives of N_{d-1}(1)
def N0(d):
    return factorial(2*d-2)/(factorial(d)*factorial(d-1))
def N1(d):
    return factorial(2*d-2)/(factorial(d-1)*factorial(d-1))/2
def N2(d):
    return (d-2)*factorial(2*d-4)/(factorial(d-2)^2)
def NP(d, expr):
    difference = expr - 1
    return N0(d) + N1(d)*difference + 1/2 * N2(d)*difference^2 + O(difference^3)


In [ ]:
L_fmoment = u_asy^d * factorial(d) / (1+u_asy) / (1 - u_asy_r * u_asy)^d / (1 - u_asy)^(d-1) * NP(d, u_asy_r)

In [ ]:
L_fmoment_asy = L_fmoment._singularity_analysis_(n, zeta=1/4, precision=2) / catalan_shifted_asy

In [ ]:
L_fmoment_asy = L_fmoment_asy.map_coefficients(
    lambda expression:
        expression.subs({
            gamma(d-1/2): factorial(2*d-2)/factorial(d-1) * sqrt(pi) * 2^(2-2*d),
            gamma(d-3/2): factorial(2*d-4)/factorial(d-2) * sqrt(pi) * 2^(4-2*d)
        }).subs({
            factorial(d): d*(d-1)*factorial(d-2),
            factorial(d-1): (d-1)*factorial(d-2),
            factorial(2*d-2): (2*d-2)*(2*d-3)*factorial(2*d-4)
        }).factor()
).map_coefficients(factor)

In [ ]:
show(L_fmoment_asy)

# Cutting Paths

## Counting total number of paths

In [ ]:
var('s t alpha chi_k')
var('k l', domain='integer')

In [ ]:
sum((binomial(l+s-1, l) * 1/(2^(s+l) - 1)), l, 1, 3).series(s==-1, 1)

In [ ]:
gamma(s).series(s==-1, 2)

In [ ]:
(s / (2^(s+1) - 1)).series(s==-1, 1)

In [ ]:
mellin_expansion = (alpha - 1) * t^-1 + (gamma(s) * zeta(s) * (s / (2^(s+1) - 1)) * t^-s).residue(s==-1).expand(); mellin_expansion

In [ ]:
mellin_expansion = (alpha - 1) * t^-1 + (gamma(s) * zeta(s) * (s / (2^(s+1) - 1)) * t^-s).residue(s==-1).expand()
mellin_expansion_in_z = fast_callable(mellin_expansion, vars=[alpha, t])(alpha, -log(u_asy))

In [ ]:
extra_factor =  (1 - sqrt(1-4*z_asy))/2

In [ ]:
mellin_expansion_P = mellin_expansion_in_z * extra_factor +  O(Z^(-3/4))

In [ ]:
mellin_expansion_P.show()

In [ ]:
coefficient_growth = mellin_expansion_P._singularity_analysis_(n, zeta=1/4, precision=3) / catalan_shifted_asy
coefficient_growth = coefficient_growth.map_coefficients(factor)
coefficient_growth.show()

In [ ]:
error = t^(3/2)
error_translated = (fast_callable(error, vars=['t'])(-log(u_asy))* extra_factor)._singularity_analysis_(n, zeta=1/4, precision=0)
(error_translated / catalan_shifted_asy).show()

In [ ]:
(sum((binomial(l+s-1, l) * 1/(2^(s+l) - 1)), l, 1, 3)*zeta(s)*gamma(s)).residue(s==-1+chi(k)).factor().show()

In [ ]:
fluc_contribution = fast_callable((s / (2^(s+1) - 1) * zeta(s) * gamma(s) * t^-s).residue(s==-1+chi(k)),
                                  vars=[t, k])(-log(u_asy), k)*extra_factor + O(Z^(-3/4 + chi(k)/2))
fluc_contribution.show()

In [ ]:
fluc_asy = fluc_contribution._singularity_analysis_(n, zeta=1/4, precision=1) / catalan_shifted_asy
fluc_asy = fluc_asy.map_coefficients(factor)
fluc_asy.show()

### Numerical verification

In [ ]:
alpha_num = sum(1/(2^k - 1) for k in srange(1, 100))

In [ ]:
def exact_coef(n):
    return sum((
        binomial(2*n-2, n-1-l*(2^r-1))
        - binomial(2*n-2, n-1-l*2^r)
        - binomial(2*n-2, n-2-l*(2^r-1))
        + binomial(2*n-2, n-2-l*2^r)
        for l in srange(0, n) for r in srange(1, ceil(log(n+1))+1))
    )

In [ ]:
time coeffs = [exact_coef(k)/catalan_number(k-1) if k > 0 else 0 for k in range(500)]

In [ ]:
list_plot(coeffs) + plot(x*(alpha_num-1), (x, 1, 500), color='red')

In [ ]:
list_plot([coeffs[k] - k*(alpha_num-1) for k in srange(1, 300)]) + plot(1/6*log(x,4), (x, 0.1, 300), color='red')

In [ ]:
abs_const = - 1/12*(euler_gamma + 4*alpha_num*log(2) - 3*log(2) + 24*zetaderiv(1, -1) + 2)/log(2)
abs_const.n()

In [ ]:
list_plot([coeffs[k] - k*(alpha_num-1) - log(k,4)/6 - abs_const.n() for k in srange(1, 500)])

In [ ]:
def delta(x):
    return sum((4*sqrt(pi)*(chi(k) - 1)*gamma(chi(k) - 1)*zeta(chi(k) - 1)/(log(2)*gamma(1/2*chi(k) - 1/2)))*exp(2*k*pi*I*x) for k in srange(-5,0)+srange(1,6))

In [ ]:
time delta_list = [0] + [delta(log(k,4)).n().real_part() for k in srange(1,500)]

In [ ]:
list_plot([coeffs[k] - k*(alpha_num-1) - log(k)/(12*log(2)) - abs_const.n() - delta_list[k]*(1 - 1/sqrt(k)) for k in srange(1, 500)])

## Cutting old leaves

In [ ]:
var('b_r_d0 b_r_d1 b_r_d2 b_r_d3 b_r_d4 B_rv')
SR_restr = SR.subring(accepting_variables=(r, alpha, d, b_r_d0, b_r_d1, b_r_d2, b_r_d3, b_r_d4))
A_func.<Z> = AsymptoticRing('Z^QQ * log(Z)^QQ', SR_restr)
z_asy = 1/4*(1-Z^-1)

def B_r_asy(expr):
    return sum(SR_restr('b_r_d{}'.format(j)) * (expr-1/4)^j/j.factorial() for j in srange(5)) + O((expr-1/4)^5)

def G_r(z, v):
    return 1/2 * (1 - sqrt(1 - 4*z*v*(B_rv*(1-v) + v)))

In [ ]:
OL_expectation_expansion = fast_callable(G_r(z, v).diff(v, 1)(v=1), vars=[z, B_rv])(z_asy, B_r_asy(z_asy))
OL_expectation_expansion = OL_expectation_expansion.map_coefficients(lambda ex: ex.factor())
OL_expectation_expansion.show()

In [ ]:
OL_expectation_asy = OL_expectation_expansion._singularity_analysis_(n, zeta=1/4, precision=3) / catalan_shifted_asy
OL_expectation_asy = OL_expectation_asy.map_coefficients(lambda ex: ex.factor())
OL_expectation_asy.show()

In [ ]:
G_r(z, v).diff(v, 2)(v=1).show()

In [ ]:
OL_fmoment2_expansion = fast_callable(G_r(z, v).diff(v, 2)(v=1), vars=[z, B_rv])(z_asy, B_r_asy(z_asy))
OL_fmoment2_expansion = OL_fmoment2_expansion.map_coefficients(lambda ex: ex.factor())
OL_fmoment2_expansion.show()

In [ ]:
OL_fmoment2_asy = OL_fmoment2_expansion._singularity_analysis_(n, zeta=1/4, precision=3) / catalan_shifted_asy
OL_fmoment2_asy = OL_fmoment2_asy.map_coefficients(lambda ex: ex.factor())
OL_fmoment2_asy.show()

In [ ]:
OL_variance_asy = OL_fmoment2_asy + OL_expectation_asy - OL_expectation_asy^2
OL_variance_asy = OL_variance_asy.map_coefficients(lambda ex: ex.factor())
OL_variance_asy.show()